# Sudoku Solver - Dancing Links / Algorithm X (Python)

Ce notebook implemente un solveur de Sudoku utilisant l'algorithme **Dancing Links (DLX)** de Donald Knuth.
C'est l'equivalent Python du notebook C# `Sudoku-5-DancingLinks.ipynb`.

## Table des matieres

1. [Introduction theorique](#1-introduction-theorique)
2. [Le probleme de couverture exacte](#2-le-probleme-de-couverture-exacte)
3. [Sudoku comme probleme de couverture exacte](#3-sudoku-comme-probleme-de-couverture-exacte)
4. [L'algorithme X de Knuth](#4-lalgorithme-x-de-knuth)
5. [Implementation Dancing Links](#5-implementation-dancing-links)
6. [Tests et benchmarks](#6-tests-et-benchmarks)

## References

- [Dancing Links - Donald Knuth (2000)](https://arxiv.org/abs/cs/0011047)
- [Exact Cover Problem - Wikipedia](https://en.wikipedia.org/wiki/Exact_cover)
- [Algorithm X - Wikipedia](https://en.wikipedia.org/wiki/Knuth%27s_Algorithm_X)

## 1. Introduction theorique

### Qu'est-ce que Dancing Links?

**Dancing Links (DLX)** est une technique elegante inventee par Donald Knuth pour implementer
efficacement son **Algorithm X**, qui resout le probleme de **couverture exacte**.

Le nom "Dancing Links" vient de la facon dont les pointeurs "dansent" lors des operations
de suppression et restauration dans une liste doublement chainee circulaire.

### Pourquoi DLX est-il efficace pour Sudoku?

| Algorithme | Complexite | Avantage |
|------------|------------|----------|
| Backtracking simple | O(9^81) pire cas | Simple a implementer |
| Backtracking + MRV | O(9^m) m=cases vides | Bonne heuristique |
| **Dancing Links** | O(n) operations par noeud | Optimal pour couverture exacte |

DLX est particulierement adapte car:
1. **Suppression/Restauration O(1)** : Grace aux listes doublement chainees
2. **Pas de copie de donnees** : Les noeuds sont simplement "deconnectes" puis "reconnectes"
3. **Heuristique MRV integree** : Choix de la colonne avec le moins de 1s

## 2. Le probleme de couverture exacte

### Definition

Etant donne:
- Un ensemble **U** = {1, 2, 3, ..., n} d'elements
- Une collection **S** = {S1, S2, ..., Sm} de sous-ensembles de U

Trouver une **couverture exacte**: une sous-collection S* de S telle que chaque element
de U appartient a **exactement un** sous-ensemble de S*.

### Exemple simple

```
U = {1, 2, 3, 4, 5, 6, 7}

S = {
  A = {1, 4, 7}
  B = {1, 4}
  C = {4, 5, 7}
  D = {3, 5, 6}
  E = {2, 3, 6, 7}
  F = {2, 7}
}
```

**Solution**: S* = {B, D, F} car:
- B couvre {1, 4}
- D couvre {3, 5, 6}
- F couvre {2, 7}
- Union = {1, 2, 3, 4, 5, 6, 7} = U (chaque element exactement une fois)

### Representation matricielle

On represente le probleme par une matrice binaire:
- Chaque **ligne** = un sous-ensemble
- Chaque **colonne** = un element de U
- M[i,j] = 1 si l'element j est dans le sous-ensemble i

```
     1  2  3  4  5  6  7
A = [1, 0, 0, 1, 0, 0, 1]
B = [1, 0, 0, 1, 0, 0, 0]
C = [0, 0, 0, 1, 1, 0, 1]
D = [0, 0, 1, 0, 1, 1, 0]
E = [0, 1, 1, 0, 0, 1, 1]
F = [0, 1, 0, 0, 0, 0, 1]
```

Une couverture exacte = selection de lignes ou chaque colonne a exactement un 1.

## 3. Sudoku comme probleme de couverture exacte

### Les 4 types de contraintes du Sudoku

Un Sudoku 9x9 standard a **324 contraintes** (colonnes) reparties en 4 categories:

| Type | Description | Nombre | Colonnes |
|------|-------------|--------|----------|
| **Cell** | Chaque cellule contient exactement un chiffre | 81 | 0-80 |
| **Row** | Chaque ligne contient chaque chiffre 1-9 | 81 | 81-161 |
| **Column** | Chaque colonne contient chaque chiffre 1-9 | 81 | 162-242 |
| **Box** | Chaque bloc 3x3 contient chaque chiffre 1-9 | 81 | 243-323 |

### Les 729 possibilites (lignes)

Chaque ligne de la matrice represente le placement d'un chiffre v (1-9) 
dans une cellule (r, c):

- **729 lignes** = 9 lignes x 9 colonnes x 9 valeurs
- Chaque ligne a exactement **4 bits a 1** (une contrainte de chaque type)

### Calcul des indices de colonnes

Pour un placement (row=r, col=c, value=v):

```python
# Contrainte Cell: cellule (r,c) est remplie
cell_col = r * 9 + c                           # 0-80

# Contrainte Row: ligne r contient valeur v
row_col = 81 + r * 9 + (v - 1)                 # 81-161

# Contrainte Column: colonne c contient valeur v  
col_col = 162 + c * 9 + (v - 1)                # 162-242

# Contrainte Box: bloc b contient valeur v
box = (r // 3) * 3 + (c // 3)
box_col = 243 + box * 9 + (v - 1)              # 243-323
```

### Exemple visuel

Placer le chiffre **5** en position **(2, 3)** (ligne 2, colonne 3):

```
Contrainte Cell:   colonne 2*9+3 = 21
Contrainte Row:    colonne 81 + 2*9 + 4 = 103  
Contrainte Column: colonne 162 + 3*9 + 4 = 193
Contrainte Box:    colonne 243 + 0*9 + 4 = 247  (bloc 0)

Ligne de la matrice: [0...1...0] avec des 1 aux positions 21, 103, 193, 247
```

## 4. L'algorithme X de Knuth

### Pseudo-code

```
function solve(matrix):
    if matrix is empty:
        return SUCCESS  # Solution trouvee!
    
    # Choisir la colonne c avec le moins de 1s (heuristique MRV)
    c = column_with_minimum_ones(matrix)
    
    if c has no 1s:
        return FAILURE  # Impasse
    
    # Pour chaque ligne r ayant un 1 dans la colonne c
    for each row r where matrix[r][c] == 1:
        # Ajouter r a la solution partielle
        solution.add(r)
        
        # Couvrir: supprimer c et toutes les lignes en conflit
        cover(c)
        for each column j where matrix[r][j] == 1:
            cover(j)
        
        # Recursion
        result = solve(reduced_matrix)
        if result == SUCCESS:
            return SUCCESS
        
        # Backtrack: restaurer les colonnes
        for each column j where matrix[r][j] == 1 (reverse order):
            uncover(j)
        uncover(c)
        solution.remove(r)
    
    return FAILURE
```

### L'astuce des Dancing Links

Dans une liste doublement chainee, supprimer un noeud x:

```python
x.left.right = x.right
x.right.left = x.left
```

Et le restaurer (x garde ses pointeurs!):

```python
x.left.right = x
x.right.left = x
```

C'est cette propriete qui permet un backtracking tres efficace: les noeuds
"dansent" en entrant et sortant de la structure sans etre detruits.

In [ ]:
# Imports
import time
from typing import List, Optional, Set, Tuple, Generator
from dataclasses import dataclass, field

print("Imports OK")

## 5. Implementation Dancing Links

### 5.1 Structure de donnees

La structure DLX utilise des noeuds doublement chaines dans 4 directions:
- **left/right**: navigation horizontale dans une ligne
- **up/down**: navigation verticale dans une colonne

Chaque colonne a un noeud special "header" qui contient:
- Le nombre de 1s dans la colonne (pour l'heuristique MRV)
- Un identifiant de colonne

Tous les headers sont lies horizontalement, avec un noeud "root" special.

In [ ]:
class DancingLinksNode:
    """Noeud dans la structure Dancing Links.
    
    Chaque noeud est connecte a 4 voisins (gauche, droite, haut, bas)
    formant une grille de listes doublement chainees circulaires.
    
    Attributes:
        left: Noeud a gauche dans la meme ligne
        right: Noeud a droite dans la meme ligne
        up: Noeud au-dessus dans la meme colonne
        down: Noeud en-dessous dans la meme colonne
        column: Reference vers le header de colonne
        row_id: Identifiant de la ligne (pour reconstruire la solution)
    """
    __slots__ = ['left', 'right', 'up', 'down', 'column', 'row_id']
    
    def __init__(self):
        self.left = self
        self.right = self
        self.up = self
        self.down = self
        self.column = None
        self.row_id = None


class ColumnHeader(DancingLinksNode):
    """Header de colonne avec compteur de taille.
    
    Attributes:
        size: Nombre de noeuds (1s) dans cette colonne
        name: Identifiant de la colonne (pour debug)
    """
    __slots__ = ['size', 'name']
    
    def __init__(self, name: int = 0):
        super().__init__()
        self.size = 0
        self.name = name
        self.column = self  # Un header pointe vers lui-meme


print("Classes DancingLinksNode et ColumnHeader definies")

### 5.2 Classe DancingLinks

Cette classe implemente l'algorithme complet:
1. **Construction** de la matrice a partir des contraintes
2. **Cover/Uncover** pour la suppression/restauration efficace
3. **Search** (Algorithm X) avec backtracking

In [ ]:
class DancingLinks:
    """Implementation complete de l'algorithme Dancing Links.
    
    Cette classe construit une matrice creuse et implemente l'Algorithm X
    de Knuth pour resoudre le probleme de couverture exacte.
    """
    
    def __init__(self, num_columns: int):
        """Initialise la structure avec le nombre de colonnes.
        
        Args:
            num_columns: Nombre de contraintes (324 pour Sudoku 9x9)
        """
        # Creer le noeud racine
        self.root = ColumnHeader(-1)
        
        # Creer les headers de colonnes
        self.columns: List[ColumnHeader] = []
        prev = self.root
        
        for i in range(num_columns):
            header = ColumnHeader(i)
            self.columns.append(header)
            
            # Lier horizontalement
            header.left = prev
            header.right = self.root
            prev.right = header
            self.root.left = header
            prev = header
        
        # Solution courante (liste d'indices de lignes)
        self.solution: List[int] = []
        self.solutions_found: List[List[int]] = []
    
    def add_row(self, row_id: int, columns: List[int]):
        """Ajoute une ligne avec des 1s aux colonnes specifiees.
        
        Args:
            row_id: Identifiant unique de la ligne
            columns: Liste des indices de colonnes ayant un 1
        """
        if not columns:
            return
        
        first_node = None
        prev_node = None
        
        for col_idx in columns:
            # Creer un nouveau noeud
            node = DancingLinksNode()
            node.row_id = row_id
            node.column = self.columns[col_idx]
            
            # Inserer en bas de la colonne
            col_header = self.columns[col_idx]
            node.up = col_header.up
            node.down = col_header
            col_header.up.down = node
            col_header.up = node
            col_header.size += 1
            
            # Lier horizontalement
            if first_node is None:
                first_node = node
                node.left = node
                node.right = node
            else:
                node.left = prev_node
                node.right = first_node
                prev_node.right = node
                first_node.left = node
            
            prev_node = node
    
    def cover(self, col: ColumnHeader):
        """Couvre une colonne (la supprime temporairement).
        
        Cette operation:
        1. Deconnecte le header de la liste des colonnes
        2. Pour chaque ligne ayant un 1 dans cette colonne,
           deconnecte tous les autres noeuds de cette ligne
        
        Args:
            col: Header de la colonne a couvrir
        """
        # Deconnecter le header horizontalement
        col.right.left = col.left
        col.left.right = col.right
        
        # Parcourir vers le bas dans la colonne
        i = col.down
        while i != col:
            # Parcourir vers la droite dans la ligne
            j = i.right
            while j != i:
                # Deconnecter verticalement
                j.down.up = j.up
                j.up.down = j.down
                j.column.size -= 1
                j = j.right
            i = i.down
    
    def uncover(self, col: ColumnHeader):
        """Decouvre une colonne (la restaure).
        
        Operation inverse de cover(), executee dans l'ordre inverse.
        
        Args:
            col: Header de la colonne a restaurer
        """
        # Parcourir vers le haut dans la colonne
        i = col.up
        while i != col:
            # Parcourir vers la gauche dans la ligne
            j = i.left
            while j != i:
                # Reconnecter verticalement
                j.column.size += 1
                j.down.up = j
                j.up.down = j
                j = j.left
            i = i.up
        
        # Reconnecter le header horizontalement
        col.right.left = col
        col.left.right = col
    
    def choose_column(self) -> Optional[ColumnHeader]:
        """Choisit la colonne avec le moins de 1s (heuristique MRV).
        
        Returns:
            Le header de colonne avec le minimum de noeuds,
            ou None si toutes les colonnes sont couvertes.
        """
        min_size = float('inf')
        best_col = None
        
        col = self.root.right
        while col != self.root:
            if col.size < min_size:
                min_size = col.size
                best_col = col
            col = col.right
        
        return best_col
    
    def search(self, find_all: bool = False) -> bool:
        """Algorithme X recursif.
        
        Args:
            find_all: Si True, trouve toutes les solutions
        
        Returns:
            True si une solution est trouvee
        """
        # Cas de base: toutes les colonnes couvertes
        if self.root.right == self.root:
            self.solutions_found.append(self.solution.copy())
            return not find_all  # Continuer si on cherche toutes les solutions
        
        # Choisir la colonne avec le moins de 1s
        col = self.choose_column()
        
        # Si une colonne est vide, impasse
        if col is None or col.size == 0:
            return False
        
        # Couvrir cette colonne
        self.cover(col)
        
        # Essayer chaque ligne ayant un 1 dans cette colonne
        row = col.down
        while row != col:
            # Ajouter cette ligne a la solution
            self.solution.append(row.row_id)
            
            # Couvrir toutes les colonnes de cette ligne
            j = row.right
            while j != row:
                self.cover(j.column)
                j = j.right
            
            # Recursion
            if self.search(find_all):
                if not find_all:
                    return True
            
            # Backtrack
            self.solution.pop()
            
            # Decouvrir les colonnes dans l'ordre inverse
            j = row.left
            while j != row:
                self.uncover(j.column)
                j = j.left
            
            row = row.down
        
        # Decouvrir la colonne choisie
        self.uncover(col)
        
        return len(self.solutions_found) > 0


print("Classe DancingLinks definie")

### 5.3 Solveur de Sudoku avec DLX

Cette classe traduit un puzzle Sudoku en probleme de couverture exacte,
puis utilise DancingLinks pour le resoudre.

In [ ]:
class SudokuGrid:
    """Representation d'une grille de Sudoku 9x9."""
    
    def __init__(self, cells: Optional[List[List[int]]] = None):
        if cells is None:
            self.cells = [[0] * 9 for _ in range(9)]
        else:
            self.cells = [row[:] for row in cells]
    
    @classmethod
    def from_string(cls, s: str) -> 'SudokuGrid':
        """Cree une grille depuis une chaine de 81 caracteres."""
        s = s.replace('.', '0').replace(' ', '').replace('\n', '')
        if len(s) != 81:
            raise ValueError(f"Attendu 81 caracteres, recu {len(s)}")
        
        grid = cls()
        for i in range(81):
            grid.cells[i // 9][i % 9] = int(s[i])
        return grid
    
    def clone(self) -> 'SudokuGrid':
        return SudokuGrid(self.cells)
    
    def is_valid(self) -> bool:
        """Verifie si la grille est une solution valide."""
        # Verifier que toutes les cellules sont remplies
        for r in range(9):
            for c in range(9):
                if self.cells[r][c] == 0:
                    return False
        
        # Verifier lignes
        for r in range(9):
            if len(set(self.cells[r])) != 9:
                return False
        
        # Verifier colonnes
        for c in range(9):
            if len(set(self.cells[r][c] for r in range(9))) != 9:
                return False
        
        # Verifier blocs
        for br in range(3):
            for bc in range(3):
                block = []
                for r in range(3):
                    for c in range(3):
                        block.append(self.cells[br*3+r][bc*3+c])
                if len(set(block)) != 9:
                    return False
        
        return True
    
    def __str__(self) -> str:
        lines = []
        for r in range(9):
            if r > 0 and r % 3 == 0:
                lines.append('-' * 21)
            row_str = ''
            for c in range(9):
                if c > 0 and c % 3 == 0:
                    row_str += '| '
                val = self.cells[r][c]
                row_str += (str(val) if val != 0 else '.') + ' '
            lines.append(row_str)
        return '\n'.join(lines)


class DLXSudokuSolver:
    """Solveur de Sudoku utilisant Dancing Links.
    
    Traduit le Sudoku en probleme de couverture exacte:
    - 324 colonnes (contraintes)
    - Jusqu'a 729 lignes (possibilites)
    - 4 bits a 1 par ligne
    """
    
    NUM_COLUMNS = 324  # 4 * 81 contraintes
    
    def __init__(self):
        self.call_count = 0
    
    def solve(self, puzzle: SudokuGrid) -> Optional[SudokuGrid]:
        """Resout le Sudoku avec Dancing Links.
        
        Args:
            puzzle: Grille de depart (0 = case vide)
        
        Returns:
            Grille resolue ou None si pas de solution
        """
        dlx = DancingLinks(self.NUM_COLUMNS)
        row_info = {}  # row_id -> (r, c, v)
        row_id = 0
        
        # Construire la matrice
        for r in range(9):
            for c in range(9):
                cell_value = puzzle.cells[r][c]
                
                # Determiner les valeurs possibles
                if cell_value != 0:
                    # Cellule deja remplie: une seule possibilite
                    values = [cell_value]
                else:
                    # Cellule vide: toutes les valeurs 1-9
                    values = range(1, 10)
                
                for v in values:
                    # Calculer les 4 colonnes a mettre a 1
                    columns = self._get_columns(r, c, v)
                    
                    # Ajouter la ligne
                    dlx.add_row(row_id, columns)
                    row_info[row_id] = (r, c, v)
                    row_id += 1
        
        # Resoudre
        if dlx.search():
            # Reconstruire la solution
            result = puzzle.clone()
            for rid in dlx.solutions_found[0]:
                r, c, v = row_info[rid]
                result.cells[r][c] = v
            return result
        
        return None
    
    def _get_columns(self, r: int, c: int, v: int) -> List[int]:
        """Calcule les 4 indices de colonnes pour un placement.
        
        Args:
            r: Ligne (0-8)
            c: Colonne (0-8)
            v: Valeur (1-9)
        
        Returns:
            Liste de 4 indices de colonnes
        """
        box = (r // 3) * 3 + (c // 3)
        
        return [
            r * 9 + c,                    # Cell constraint (0-80)
            81 + r * 9 + (v - 1),         # Row constraint (81-161)
            162 + c * 9 + (v - 1),        # Column constraint (162-242)
            243 + box * 9 + (v - 1)       # Box constraint (243-323)
        ]


print("Classes SudokuGrid et DLXSudokuSolver definies")

## 6. Tests et benchmarks

### 6.1 Test basique

In [ ]:
# Puzzle de test
test_puzzle = "530070000600195000098000060800060003400803001700020006060000280000419005000080079"

puzzle = SudokuGrid.from_string(test_puzzle)
print("Puzzle initial:")
print(puzzle)
print()

# Resoudre
solver = DLXSudokuSolver()
start = time.time()
solution = solver.solve(puzzle)
elapsed = (time.time() - start) * 1000

if solution:
    print(f"Solution trouvee en {elapsed:.2f} ms:")
    print(solution)
    print(f"\nSolution valide: {solution.is_valid()}")
else:
    print("Pas de solution!")

### 6.2 Chargement des puzzles depuis fichiers

In [ ]:
def load_puzzles(filepath: str, max_puzzles: int = None) -> List[str]:
    """Charge les puzzles depuis un fichier.
    
    Chaque ligne du fichier doit contenir au moins 81 caracteres
    representant un puzzle (0 ou . pour les cases vides).
    
    Args:
        filepath: Chemin vers le fichier
        max_puzzles: Nombre maximum de puzzles a charger
    
    Returns:
        Liste de chaines de 81 caracteres
    """
    puzzles = []
    try:
        with open(filepath, 'r') as f:
            for line in f:
                line = line.strip()
                if len(line) >= 81:
                    puzzles.append(line[:81])
                    if max_puzzles and len(puzzles) >= max_puzzles:
                        break
    except FileNotFoundError:
        print(f"Fichier non trouve: {filepath}")
    return puzzles

# Charger les puzzles
easy_puzzles = load_puzzles('Puzzles/Sudoku_Easy51.txt', max_puzzles=10)
hard_puzzles = load_puzzles('Puzzles/Sudoku_hardest.txt')

print(f"Puzzles faciles charges: {len(easy_puzzles)}")
print(f"Puzzles difficiles charges: {len(hard_puzzles)}")

### 6.3 Benchmark sur plusieurs puzzles

In [ ]:
def benchmark_dlx(puzzles: List[str], name: str):
    """Benchmark le solveur DLX sur une liste de puzzles.
    
    Args:
        puzzles: Liste de puzzles (chaines de 81 caracteres)
        name: Nom du benchmark pour l'affichage
    """
    if not puzzles:
        print(f"Aucun puzzle pour {name}")
        return
    
    print(f"\n{'='*60}")
    print(f"Benchmark DLX: {name} ({len(puzzles)} puzzles)")
    print('='*60)
    
    solver = DLXSudokuSolver()
    total_time = 0
    solved_count = 0
    times = []
    
    for i, puzzle_str in enumerate(puzzles):
        puzzle = SudokuGrid.from_string(puzzle_str)
        
        start = time.time()
        solution = solver.solve(puzzle)
        elapsed = (time.time() - start) * 1000
        
        total_time += elapsed
        times.append(elapsed)
        
        if solution and solution.is_valid():
            solved_count += 1
            status = "OK"
        else:
            status = "ECHEC"
        
        if i < 5 or status == "ECHEC":  # Premiers et echecs
            print(f"  Puzzle {i+1:2d}: {status} en {elapsed:>8.3f} ms")
    
    if len(puzzles) > 5 and solved_count == len(puzzles):
        print(f"  ... ({len(puzzles)-5} autres puzzles resolus)")
    
    print(f"\nResultats:")
    print(f"  Resolus:     {solved_count}/{len(puzzles)}")
    print(f"  Temps total: {total_time:>10.2f} ms")
    print(f"  Temps moyen: {total_time/len(puzzles):>10.3f} ms")
    print(f"  Temps min:   {min(times):>10.3f} ms")
    print(f"  Temps max:   {max(times):>10.3f} ms")

# Benchmarks
benchmark_dlx(easy_puzzles, "Puzzles Faciles")
benchmark_dlx(hard_puzzles, "Puzzles Difficiles (Top 11)")

## 7. Conclusion et comparaison

### Performances attendues

| Algorithme | Puzzle facile | Puzzle difficile |
|------------|---------------|------------------|
| Backtracking simple | 1-10 ms | 100-1000 ms |
| Backtracking + MRV | 0.5-5 ms | 10-100 ms |
| **Dancing Links** | 0.1-1 ms | 1-10 ms |
| OR-Tools CP-SAT | 1-5 ms | 5-20 ms |
| Z3 SMT | 5-20 ms | 20-100 ms |

### Avantages de Dancing Links

1. **Extremement rapide** pour les problemes de couverture exacte
2. **Pas de copie de donnees** lors du backtracking
3. **Heuristique MRV integree** naturellement
4. **Implementation elegante** et educative

### Inconvenients

1. **Complexite d'implementation** plus elevee
2. **Specifique** aux problemes de couverture exacte
3. **Memoire** utilisee pour les pointeurs

### Navigation

- **Notebook C# equivalent**: [Sudoku-5-DancingLinks.ipynb](Sudoku-5-DancingLinks.ipynb)
- **Precedent**: [Sudoku-Python-Genetic.ipynb](Sudoku-Python-Genetic.ipynb)
- **Voir aussi**: [Sudoku-Python-ORTools-Z3.ipynb](Sudoku-Python-ORTools-Z3.ipynb)